第一轮矩阵
投票模型，投出模型


投票模型：
deepseekv3:./small_sample_output_dir/split0_output_summary_voted_processed_five_models_deepseek-chat_voted_five_models_new.json
yi- Lightning：./small_sample_output_dir/split0_output_summary_voted_processed_five_models_yi-lightning_voted_five_models.json
gpt-4o：./small_sample_output_dir/split0_output_summary_voted_processed_five_models_gpt-4o_voted_five_models_1.json
o1-mini:./small_sample_output_dir/split0_output_summary_voted_processed_five_models_o1-mini_voted_five_models.json
qwen-max:./small_sample_output_dir/split0_output_summary_voted_processed_five_models_qwen-max_voted_five_models_final.json

In [1]:
import os
import json

In [ ]:
voting_models_results_file_paths = {
    'deepseekv3':'./small_sample_output_dir/split0_output_summary_voted_processed_five_models_deepseek-chat_voted_five_models_new.json',
    'yi-lightning':'./small_sample_output_dir/split0_output_summary_voted_processed_five_models_yi-lightning_voted_five_models.json',
    'gpt-4o':'./small_sample_output_dir/split0_output_summary_voted_processed_five_models_gpt-4o_voted_five_models_1.json',
    'o1-mini':'./small_sample_output_dir/split0_output_summary_voted_processed_five_models_o1-mini_voted_five_models.json',
    'qwen-max':'./small_sample_output_dir/split0_output_summary_voted_processed_five_models_qwen-max_voted_five_models_final.json'
}

In [3]:
with open(voting_models_results_file_paths['deepseekv3'],'r') as f:
    data = json.load(f)

data[0]

{'question': '@org.junit.Test\npublic void calculateChangeInPriceSameNumbersShouldReturnZero() throws java.lang.Exception {\n    ru.unn.agile.ElasticityOfDemand.ElasticityOfDemandCalculator calculator = new ru.unn.agile.ElasticityOfDemand.ElasticityOfDemandCalculator();\n    double result = calculator.calculateChangeInPrice(java.math.BigDecimal.ONE, java.math.BigDecimal.ONE);\n    assertEquals(0, result, 0);\n}',
 'function_description': {'A': '测试方法验证相同价格变动计算返回零。',
  'B': '测试价格变化计算返回零的功能。',
  'C': '这是一个单元测试，验证当价格变化前后相同时，价格变化率应为零。',
  'D': '测试当商品价格和数量相同的情况下，价格变动率为零。 \n',
  'E': '测试相同输入时价格变化应返回零。'},
 'vulnerability_analysis': {'A': None,
  'B': None,
  'C': "[\n   {\n      'CWE_Type': 'CWE-20',\n      'CWE_Code': 'calculator.calculateChangeInPrice(java.math.BigDecimal.ONE, java.math.BigDecimal.ONE)',\n      'CWE_Description': '未对输入参数进行边界值和异常值校验，如零值、负值或极大值，可能导致计算错误或异常。建议添加输入参数验证。'\n   }\n]\n\n这个测试用例虽然测试了相等值的情况，但从安全角度来看，缺少对输入参数的有效性验证。在实际应用中，价格计算需要考虑各种边界情况和异常输入，以确保计算结果的准确性和系统的稳定性。",
  'E': 

In [20]:
def get_function_description_voted_answer(data):
    import re
    answer = re.search(r'所以我的答案是：(None|\w)', data)
    if answer:
        extracted_answer = answer.group(1)  # 获取匹配的答案 (A/B/C/D/None)
    else:
        extracted_answer = None
        # print("未能找到预期格式的答案")

    return extracted_answer

get_function_description_voted_answer(data[499]['function_description_voted_answer'])

'D'

In [44]:
# 建立一个矩阵，数据量共500，共有5个投票模型，则横轴为1-500，纵轴为5个投票模型，矩阵的值为每个模型对每个样本的投票结果，0表示None，1表示A，2表示B，3表示C，4表示D，5表示E
import numpy as np
voting_matrix = np.zeros((len(voting_models_results_file_paths), len(data)))
for model_index, model_name in enumerate(voting_models_results_file_paths):
    with open(voting_models_results_file_paths[model_name], 'r') as f:
        data = json.load(f)
    for index in range(len(data)):
        # 使用get_function_description_voted_answer函数获取每个模型的投票结果
        try:
            answer = get_function_description_voted_answer(data[index]['function_description_voted_answer'])
        except Exception as e:
            answer = 'None'
        if answer == 'None':
            voting_matrix[model_index][index] = 0
        elif answer == None:
            voting_matrix[model_index][index] = 0
        else:
            # 将答案转换为数字
            answer_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
            voting_matrix[model_index][index] = answer_to_number[answer]

# 打印投票矩阵
print(voting_matrix)


[[4. 4. 1. ... 1. 4. 2.]
 [3. 3. 3. ... 4. 2. 4.]
 [4. 2. 5. ... 4. 2. 2.]
 [4. 4. 4. ... 0. 0. 0.]
 [5. 0. 5. ... 4. 2. 4.]]


In [21]:
# 第二个矩阵，是一个三维的矩阵，横轴为500，纵轴为5，深度为5，纵轴表示被投出的5个模型，如A,B,C,D,E，深度表示5个投票模型，如deepseekv3,yi-lightning,gpt-4o,o1-mini,qwen-max，矩阵的值为0或1，表示该模型是否被投出，如deepseekv3在第一条数据上投了A，则voting_matrix[0][0][0] = 1，其他为0，其他模型同理,如果答案为None，则表示该模型没有投出，矩阵的值为0。
import numpy as np


voting_matrix_3d = np.zeros((5, 5, 500))



models = ['deepseekv3', 'yi-lightning', 'gpt-4o', 'o1-mini', 'qwen-max']
answers = ['A', 'B', 'C', 'D', 'E']
answer_to_index = {answer: i for i, answer in enumerate(answers)}

# 填充矩阵
for model_index, model in enumerate(models):
    with open(voting_models_results_file_paths[model], 'r') as f:
        data = json.load(f)
    for index, sample in enumerate(data):
        # print(sample)
        try:
            # print(sample['function_description_voted_answer'])
            answer = get_function_description_voted_answer(sample['function_description_voted_answer'])
        except Exception as e:
            # print(e)
            answer = None
        # print(answer)
        if answer in answer_to_index:
            answer_index = answer_to_index[answer]
            # print(model_index,index,answer_index)
            # break
    
            voting_matrix_3d[model_index][answer_index][index] = 1
            # break
    # break

print(voting_matrix_3d)

[[[0. 0. 1. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 1. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 1. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 1. 0. 1.]
  [1. 0. 1. ... 0. 0. 0.]]]


# 第二轮投票
在本轮需要考虑可能会投出两个模型

In [22]:
def get_cwe_list(text):
    cwe_list = []
    import re
    if text is None:
        return []
    cwe_pattern = re.compile(r'CWE-(?:\d+|other)')
    cwe_list = cwe_pattern.findall(text)
    return cwe_list

def get_vulnerability_voting_result(text):
    import re
    answer = re.search(r'所以我的答案是：(None|[\w,]+)！', text)
    if answer:
        extracted_answer = answer.group(1)  
    else:
        extracted_answer = None

    return extracted_answer

get_vulnerability_voting_result(data[0]['vulnerability_analysis_voted_answer'])

'B,C'

In [ ]:
# 建立一个矩阵，数据量共500，共有5个投票模型，则横轴为1-500，纵轴为5个投票模型，矩阵的值为每个模型对每个样本的投票结果，0表示None，1表示A，2表示B，3表示C，4表示D，5表示E
import numpy as np
voting_matrix = np.empty((5, 500), dtype=object)
for i in range(5):
    for j in range(500):
        voting_matrix[i, j] = []
for model_index, model_name in enumerate(voting_models_results_file_paths):
    with open(voting_models_results_file_paths[model_name], 'r') as f:
        data = json.load(f)
    for index in range(len(data)):
        # get_vulnerability_voting_result
        try:
            answer_str = get_vulnerability_voting_result(data[index]['vulnerability_analysis_voted_answer'])
            answer_list = answer_str.split(',')
        except Exception as e:
            answer_list = ['None']
        for answer_item in answer_list:  
            answer_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
            if answer_item == 'None':
                voting_matrix[model_index][index] = [0]
            else:
                voting_matrix[model_index][index].append(answer_to_number[answer_item])
            
print(voting_matrix)


[[list([3]) list([3]) list([2, 4, 5]) ... list([3]) list([2, 5])
  list([4])]
 [list([4]) list([1]) list([5]) ... list([0]) list([0]) list([0])]
 [list([1, 5]) list([1, 5, 3]) list([2, 3]) ... list([3, 5]) list([4])
  list([3])]
 [list([5]) list([1, 2]) list([3, 5]) ... list([0]) list([5]) list([0])]
 [list([2, 3]) list([5]) list([1, 2]) ... list([2]) list([4]) list([2])]]


In [32]:
import numpy as np
import json


NUM_MODELS = 5
NUM_ANSWERS = 5
NUM_SAMPLES = 500

MODELS = ['deepseekv3', 'yi-lightning', 'gpt-4o', 'o1-mini', 'qwen-max']
ANSWERS = ['A', 'B', 'C', 'D', 'E']
ANSWER_TO_INDEX = {answer: i for i, answer in enumerate(ANSWERS)}
MODEL_TO_INDEX = {model: i for i, model in enumerate(MODELS)}

voting_matrix_3d = np.zeros((NUM_MODELS, NUM_ANSWERS, NUM_SAMPLES))

for model_index, model_name in enumerate(MODELS):
    file_path = voting_models_results_file_paths.get(model_name)
    
    with open(file_path, 'r') as f:
        data = json.load(f)

    for sample_index, sample in enumerate(data):
        try:
            answer_str = get_vulnerability_voting_result(sample['vulnerability_analysis_voted_answer'])
            answer_list = answer_str.split(',')
        except Exception as e:
            answer_list = ['None']
        for answer_item in answer_list:
            if answer_item in ANSWER_TO_INDEX:
                answer_index = ANSWER_TO_INDEX[answer_item]
                voting_matrix_3d[model_index][answer_index][sample_index] = 1

print(voting_matrix_3d)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 1. 0.]
  [1. 1. 0. ... 1. 0. 0.]
  [0. 0. 1. ... 0. 0. 1.]
  [0. 0. 1. ... 0. 1. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[1. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 1. 1. ... 1. 0. 1.]
  [0. 0. 0. ... 0. 1. 0.]
  [1. 1. 0. ... 1. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 1. ... 0. 1. 0.]]

 [[0. 0. 1. ... 0. 0. 0.]
  [1. 0. 1. ... 1. 0. 1.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 1. 0. ... 0. 0. 0.]]]


CWE-Type矩阵

In [ ]:
with open('./small_sample_output_dir/split0_output_summary_five_models.json','r') as f:
    data = json.load(f)

data[0]

{'question': 'import sys, os, io\r\ninput = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline\r\n\r\ndef binary_search(c1, c2):\r\n    m = (c1 + c2 + 1) // 2\r\n    while abs(c1 - c2) > 1:\r\n        m = (c1 + c2 + 1) // 2\r\n        if ok(m):\r\n            c1 = m\r\n        else:\r\n            c2 = m\r\n    m = max(m - 1, 1)\r\n    while ok(m):\r\n        m += 1\r\n    return m - 1\r\n\r\ndef ok(m):\r\n    c = [0] * k\r\n    s0 = 0\r\n    for i in range(n - 1, n - m, -1):\r\n        si = s[i] - 97\r\n        if si >= 0:\r\n            c[si] += 1\r\n            s0 += 1\r\n    for i in range(n - m, -1, -1):\r\n        si = s[i] - 97\r\n        if si >= 0:\r\n            c[si] += 1\r\n            s0 += 1\r\n        u = i + m - 1\r\n        for j in range(k):\r\n            if not c[j] ^ s0:\r\n                x[j][i] = i\r\n            else:\r\n                x[j][i] = x[j][i + 1]\r\n        su = s[u] - 97\r\n        if su >= 0:\r\n            c[su] -= 1\r\n            s0 -= 1\r\n 

In [51]:
# 先统计总共出现了多少个CWE类型
statistics_data = []
model_list = ['gpt4o', 'deepseekv3', 'claude3.5-1022', 'gemini1.5-pro-latest', 'yi-lightning']
for index, item in enumerate(data):
    js = {
        'index': index,
        'question': item['question'],
        'cwe_list': {
            'gpt4o': [],
            'deepseekv3': [],
            'claude3.5-1022': [],
            'gemini1.5-pro-latest': [],
            'yi-lightning': []
        },
        'cwe_count': {
            'gpt4o': 0,
            'deepseekv3': 0,
            'claude3.5-1022': 0,
            'gemini1.5-pro-latest': 0,
            'yi-lightning': 0
        }
    }
    for model in model_list:
        cwe_list = get_cwe_list(item[model]['vulnerability_analysis'])
        js['cwe_list'][model] = cwe_list
        js['cwe_count'][model] = len(cwe_list)
    statistics_data.append(js)

In [53]:
cwe_count = {}
for item in statistics_data:
    for model in model_list:
        for cwe in item['cwe_list'][model]:
            cwe_count[cwe] = cwe_count.get(cwe, 0) + 1

cwe_count,len(cwe_count.keys())

({'CWE-20': 2297,
  'CWE-125': 1941,
  'CWE-787': 1311,
  'CWE-129': 28,
  'CWE-416': 216,
  'CWE-476': 33,
  'CWE-755': 1,
  'CWE-561': 12,
  'CWE-788': 3,
  'CWE-other': 80,
  'CWE-789': 6,
  'CWE-22': 106,
  'CWE-457': 38,
  'CWE-400': 26,
  'CWE-252': 5,
  'CWE-522': 3,
  'CWE-459': 9,
  'CWE-362': 6,
  'CWE-390': 1,
  'CWE-352': 36,
  'CWE-209': 20,
  'CWE-404': 13,
  'CWE-601': 3,
  'CWE-79': 51,
  'CWE-434': 32,
  'CWE-674': 5,
  'CWE-190': 42,
  'CWE-197': 4,
  'CWE-682': 2,
  'CWE-369': 5,
  'CWE-691': 1,
  'CWE-248': 10,
  'CWE-396': 4,
  'CWE-665': 5,
  'CWE-401': 9,
  'CWE-284': 2,
  'CWE-502': 4,
  'CWE-681': 1,
  'CWE-121': 1,
  'CWE-834': 1,
  'CWE-664': 1,
  'CWE-89': 23,
  'CWE-703': 2,
  'CWE-412': 1,
  'CWE-120': 13,
  'CWE-398': 3,
  'CWE-470': 2,
  'CWE-732': 5,
  'CWE-835': 2,
  'CWE-676': 3,
  'CWE-563': 1,
  'CWE-642': 1,
  'CWE-366': 2,
  'CWE-119': 10,
  'CWE-134': 2,
  'CWE-200': 2,
  'CWE-758': 3,
  'CWE-226': 1,
  'CWE-273': 1,
  'CWE-918': 1,
  'CWE-391': 

In [54]:
statistics_data[0]

{'index': 0,
 'question': 'import sys, os, io\r\ninput = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline\r\n\r\ndef binary_search(c1, c2):\r\n    m = (c1 + c2 + 1) // 2\r\n    while abs(c1 - c2) > 1:\r\n        m = (c1 + c2 + 1) // 2\r\n        if ok(m):\r\n            c1 = m\r\n        else:\r\n            c2 = m\r\n    m = max(m - 1, 1)\r\n    while ok(m):\r\n        m += 1\r\n    return m - 1\r\n\r\ndef ok(m):\r\n    c = [0] * k\r\n    s0 = 0\r\n    for i in range(n - 1, n - m, -1):\r\n        si = s[i] - 97\r\n        if si >= 0:\r\n            c[si] += 1\r\n            s0 += 1\r\n    for i in range(n - m, -1, -1):\r\n        si = s[i] - 97\r\n        if si >= 0:\r\n            c[si] += 1\r\n            s0 += 1\r\n        u = i + m - 1\r\n        for j in range(k):\r\n            if not c[j] ^ s0:\r\n                x[j][i] = i\r\n            else:\r\n                x[j][i] = x[j][i + 1]\r\n        su = s[u] - 97\r\n        if su >= 0:\r\n            c[su] -= 1\r\n           

In [56]:
cwe_keys = list(cwe_count.keys())
# 将cwe_keys转换为数字
cwe_keys_to_number = {cwe: i for i, cwe in enumerate(cwe_keys)}
cwe_keys

['CWE-20',
 'CWE-125',
 'CWE-787',
 'CWE-129',
 'CWE-416',
 'CWE-476',
 'CWE-755',
 'CWE-561',
 'CWE-788',
 'CWE-other',
 'CWE-789',
 'CWE-22',
 'CWE-457',
 'CWE-400',
 'CWE-252',
 'CWE-522',
 'CWE-459',
 'CWE-362',
 'CWE-390',
 'CWE-352',
 'CWE-209',
 'CWE-404',
 'CWE-601',
 'CWE-79',
 'CWE-434',
 'CWE-674',
 'CWE-190',
 'CWE-197',
 'CWE-682',
 'CWE-369',
 'CWE-691',
 'CWE-248',
 'CWE-396',
 'CWE-665',
 'CWE-401',
 'CWE-284',
 'CWE-502',
 'CWE-681',
 'CWE-121',
 'CWE-834',
 'CWE-664',
 'CWE-89',
 'CWE-703',
 'CWE-412',
 'CWE-120',
 'CWE-398',
 'CWE-470',
 'CWE-732',
 'CWE-835',
 'CWE-676',
 'CWE-563',
 'CWE-642',
 'CWE-366',
 'CWE-119',
 'CWE-134',
 'CWE-200',
 'CWE-758',
 'CWE-226',
 'CWE-273',
 'CWE-918',
 'CWE-391',
 'CWE-484',
 'CWE-778',
 'CWE-785',
 'CWE-394',
 'CWE-395',
 'CWE-78',
 'CWE-597',
 'CWE-414',
 'CWE-833',
 'CWE-367',
 'CWE-1339',
 'CWE-749',
 'CWE-704',
 'CWE-306',
 'CWE-680',
 'CWE-570',
 'CWE-23',
 'CWE-126',
 'CWE-477',
 'CWE-359',
 'CWE-798',
 'CWE-475',
 'CWE-5

In [ ]:
# 是一个三维的矩阵，横轴为86个cwe类型的数量，纵轴为5，深度为500，纵轴表示5个模型，如gpt4o,deepseekv3,claude3.5-1022,gemini1.5-pro-latest,yi-lightning，，横轴表示CWE类型，深度表示500条数据，矩阵的值为0或1，表示该模型是否是该CWE类型，如果结果中有这个CWE类型，则voting_matrix_3d[0][0][0] = 1，其他为0
import numpy as np
voting_matrix_3d = np.zeros((500, 5, 86))

MODEL_TO_INDEX = {model: i for i, model in enumerate(model_list)}

for index, item in enumerate(statistics_data):
    for model in model_list:
        for cwe in item['cwe_list'][model]:
            voting_matrix_3d[index][MODEL_TO_INDEX[model]][cwe_keys_to_number[cwe]] = 1

voting_matrix_3d

array([[[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.]],

       [[1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

  